<a href="https://colab.research.google.com/github/MINJIN82/-Ubion/blob/main/5_1_type2(blank)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 신용카드서비스를 떠나는 고객을 찾아라
- 나이, 급여, 결혼 상태, 신용 카드 한도, 신용 카드 카테고리 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1, 정밀도(Precision), 재현율(Recall)을 구하시오
- target : Attrition_Flag (1:이탈, 0:유지)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
CLIENTNUM,Attrition_Flag
788544108,0.633
719356008,0.123
712142733,0.355
~~~


### 평가(분류)
~~~
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 정확도
print(accuracy_score(y_test, pred))

# 정밀도
print(precision_score(y_test, pred))

# 재현율 (민감도)
print(recall_score(y_test, pred))

# F1
print(f1_score(y_test , pred))

# roc-auc
print(roc_auc_score(y_test, pred_proba))
~~~

## 데이터 불러오기

In [1]:
#데이터 불러오기
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
print(train.shape, test.shape)

(8101, 21) (2026, 20)


In [4]:
print(train.info(), test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 8101 non-null   int64  
 1   Customer_Age              8101 non-null   int64  
 2   Gender                    8101 non-null   object 
 3   Dependent_count           8101 non-null   int64  
 4   Education_Level           8101 non-null   object 
 5   Marital_Status            8101 non-null   object 
 6   Income_Category           8101 non-null   object 
 7   Card_Category             8101 non-null   object 
 8   Months_on_book            8101 non-null   int64  
 9   Total_Relationship_Count  8101 non-null   int64  
 10  Months_Inactive_12_mon    8101 non-null   int64  
 11  Contacts_Count_12_mon     8101 non-null   int64  
 12  Credit_Limit              8101 non-null   float64
 13  Total_Revolving_Bal       8101 non-null   int64  
 14  Avg_Open

In [5]:
print(train.head(),test.head())

   CLIENTNUM  Customer_Age Gender  Dependent_count Education_Level  \
0  778461408            38      M                1     High School   
1  718263108            44      F                3         Unknown   
2  780384483            49      M                4         Unknown   
3  719592483            43      F                5        Graduate   
4  771453033            47      M                3     High School   

  Marital_Status Income_Category Card_Category  Months_on_book  \
0        Married    $80K - $120K          Blue              23   
1       Divorced  Less than $40K          Blue              32   
2        Unknown    $80K - $120K          Blue              39   
3         Single         Unknown          Blue              34   
4        Married     $60K - $80K          Blue              35   

   Total_Relationship_Count  ...  Contacts_Count_12_mon  Credit_Limit  \
0                         4  ...                      2       22149.0   
1                         3  ...    

In [6]:
print(train.describe(), test.describe())

          CLIENTNUM  Customer_Age  Dependent_count  Months_on_book  \
count  8.101000e+03   8101.000000      8101.000000     8101.000000   
mean   7.391006e+08     46.309962         2.336502       35.904703   
std    3.686504e+07      8.042270         1.308693        8.019009   
min    7.080821e+08     26.000000         0.000000       13.000000   
25%    7.130178e+08     41.000000         1.000000       31.000000   
50%    7.179475e+08     46.000000         2.000000       36.000000   
75%    7.731407e+08     52.000000         3.000000       40.000000   
max    8.283431e+08     73.000000         5.000000       56.000000   

       Total_Relationship_Count  Months_Inactive_12_mon  \
count               8101.000000             8101.000000   
mean                   3.817060                2.343044   
std                    1.549786                1.006502   
min                    1.000000                0.000000   
25%                    3.000000                2.000000   
50%            

In [7]:
print(train.isnull().sum(), test.isnull().sum())

CLIENTNUM                   0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
Attrition_Flag              0
dtype: int64 CLIENTNUM                   0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon      

In [11]:
train['Attrition_Flag'].value_counts()

0    6815
1    1286
Name: Attrition_Flag, dtype: int64

## 데이터 전처리 & 피처엔지니어링

In [17]:
#수치 데이터와 Object 데이터 분리
n_train = train.select_dtypes(exclude="O").copy()
c_train = train.select_dtypes(include="O").copy()
n_test = test.select_dtypes(exclude="O").copy()
c_test = test.select_dtypes(include="O").copy()

train.select_dtypes(include="O").columns

Index(['Gender', 'Education_Level', 'Marital_Status', 'Income_Category',
       'Card_Category'],
      dtype='object')

In [16]:
from sklearn.preprocessing import MinMaxScaler

cols = [ 'Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']

scaler = MinMaxScaler()
n_train[cols]=scaler.fit_transform(n_train[cols])
n_test[cols]=scaler.transform(n_test[cols])

n_train

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,778461408,0.255319,0.2,0.232558,0.6,0.500000,0.333333,0.626123,0.392133,0.613073,0.480718,0.104656,0.201550,0.228571,0.045045,0
1,718263108,0.382979,0.6,0.441860,0.4,0.500000,0.333333,0.007700,0.578864,0.006751,0.270533,0.128049,0.248062,0.175714,0.861862,1
2,780384483,0.489362,0.8,0.604651,0.2,0.333333,0.333333,0.043767,0.367501,0.056732,0.278481,0.275760,0.372093,0.216571,0.321321,1
3,719592483,0.361702,1.0,0.488372,1.0,1.000000,0.500000,0.095796,0.969408,0.062701,0.190168,0.239222,0.519380,0.250857,0.530531,0
4,771453033,0.446809,0.6,0.511628,0.8,0.166667,0.333333,0.480828,0.508542,0.465332,0.386812,0.147042,0.364341,0.179714,0.074074,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,715165758,0.382979,0.6,0.511628,0.4,0.666667,0.166667,0.784598,0.233214,0.776548,0.241684,0.129887,0.356589,0.354286,0.021021,0
8097,713902983,0.340426,0.6,0.534884,0.6,0.500000,0.166667,0.007186,0.545093,0.008721,0.153665,0.214493,0.519380,0.163143,0.819820,0
8098,715937133,0.808511,0.2,0.906977,1.0,0.333333,0.333333,0.107647,0.913389,0.078144,0.210774,0.059652,0.294574,0.142857,0.460460,0
8099,715532208,0.829787,0.2,0.976744,0.4,0.500000,0.333333,0.077747,0.597537,0.072523,0.225493,0.182522,0.395349,0.161143,0.375375,0


In [20]:
from sklearn.preprocessing import LabelEncoder

cols =['Gender', 'Education_Level', 'Marital_Status', 'Income_Category',
       'Card_Category']

for col in cols:
    le = LabelEncoder()
    c_train[col] = le.fit_transform(c_train[col])
    c_test[col] = le.transform(c_test[col])

c_train

,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,1,3,1,3,0
1,0,6,0,4,0
2,1,6,3,3,0
3,0,2,2,5,0
4,1,3,1,2,0
...,...,...,...,...,...
8096,1,4,1,0,0
8097,0,3,1,4,0
8098,0,5,1,1,0
8099,0,3,2,4,0


In [21]:
train = pd.concat([n_train, c_train],axis = 1)
test = pd.concat([n_test, c_test],axis = 1)


,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,...,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,778461408,38,1,23,4,3,2,22149.0,987,21162.0,...,2409,36,0.800,0.045,0,1,3,1,3,0
1,718263108,44,3,32,3,3,2,1693.0,1457,236.0,...,2829,42,0.615,0.861,1,0,6,0,4,0
2,780384483,49,4,39,2,2,2,2886.0,925,1961.0,...,5481,58,0.758,0.321,1,1,6,3,3,0
3,719592483,43,5,34,6,6,3,4607.0,2440,2167.0,...,4825,77,0.878,0.530,0,0,2,2,5,0
4,771453033,47,3,35,5,1,2,17343.0,1280,16063.0,...,3170,57,0.629,0.074,0,1,3,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,715165758,44,3,35,3,4,1,27391.0,587,26804.0,...,2862,56,1.240,0.021,0,1,4,1,0,0
8097,713902983,42,3,36,4,3,1,1676.0,1372,304.0,...,4381,77,0.571,0.819,0,0,3,1,4,0
8098,715937133,64,1,52,6,2,2,4999.0,2299,2700.0,...,1601,48,0.500,0.460,0,0,5,1,1,0
8099,715532208,65,1,55,3,3,2,4010.0,1504,2506.0,...,3807,61,0.564,0.375,0,0,3,2,4,0


In [22]:
train = train.drop('CLIENTNUM', axis=1)
test_id = test.pop('CLIENTNUM')

## 검증 데이터 분리

In [ ]:
help(train_test_split)

In [27]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train.drop('Attrition_Flag',axis=1),
                                            train['Attrition_Flag'],
                                            test_size=0.2, random_state=2022)

X_tr.shape,X_val.shape, y_tr.shape, y_val.shape

((6480, 19), (1621, 19), (6480,), (1621,))

In [28]:
X_tr.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
6806,50,1,30,1,3,2,14217.0,1816,12401.0,0.978,15554,99,0.678,0.128,1,5,2,2,0
5635,50,2,36,5,4,1,2428.0,2005,423.0,0.835,5170,68,0.659,0.826,0,0,1,1,0
2517,43,5,32,2,3,1,11508.0,0,11508.0,1.005,5242,74,0.574,0.000,0,0,2,1,0
2717,47,3,36,4,2,2,2926.0,0,2926.0,0.676,1304,24,0.846,0.000,1,2,2,2,0
3773,63,0,53,5,3,2,2271.0,1278,993.0,0.375,1550,33,0.650,0.563,0,3,1,4,0


## 모델 & 평가

In [34]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=2022)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

In [35]:
import sklearn.metrics
print(sklearn.metrics.__all__)

['accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'ConfusionMatrixDisplay', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_tweedie_score', 'd2_absolute_error_score', 'd2_pinball_score', 'dcg_score', 'davies_bouldin_score', 'DetCurveDisplay', 'det_curve', 'DistanceMetric', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'get_scorer', 'hamming_loss', 'hinge_loss', 'homogeneity_completeness_v_measure', 'homogeneity_score', 'jaccard_score', 'label_ranking_average_precision_score', 'label_ranking_loss', 'log_loss', 'make_scorer', 'nan_euclidean_distances', 'matthews_corrcoef', 'max_error', 'mean_absolute_error', 'mean_squared_error', 'mean_squared_log_error', 'mean_pinball_loss', 'mean_poisson_

In [40]:
# ROC-AUC, 정확도(Accuracy), F1, 정밀도(Precision), 재현율(Recall)
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

print(accuracy_score(y_val, pred))
print(f1_score(y_val, pred))
print(precision_score(y_val, pred))
print(recall_score(y_val, pred))

0.9660703269586675
0.8893360160965794
0.9404255319148936
0.8435114503816794


In [41]:
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.9899257986058452


## 예측 및 csv 제출

In [42]:
pred = model.predict_proba(test)
pred

array([[1.  , 0.  ],
       [0.97, 0.03],
       [0.91, 0.09],
       ...,
       [0.78, 0.22],
       [0.99, 0.01],
       [0.88, 0.12]])

In [46]:
submit = pd.DataFrame({
                      "CLIENTNUM" : test_id,
                      "Attrition_Flag" :pred[:,1]
})
submit

,CLIENTNUM,Attrition_Flag
0,788544108,0.00
1,719356008,0.03
2,712142733,0.09
3,789394833,0.01
4,742778358,0.00
...,...,...
2021,787361583,0.03
2022,713474733,0.74
2023,708486033,0.22
2024,717683583,0.01


In [45]:
submit.to_csv("0000.csv", index=False)

In [47]:
pd.read_csv("0000.csv")

,CLIENTNUM,Attrition_Flag
0,788544108,0.00
1,719356008,0.03
2,712142733,0.09
3,789394833,0.01
4,742778358,0.00
...,...,...
2021,787361583,0.03
2022,713474733,0.74
2023,708486033,0.22
2024,717683583,0.01


##정리

In [48]:
y_test = pd.read_csv("y_test.csv")
print(roc_auc_score(y_test, pred[:,1]))

0.9878886500691804
